In [1]:
%load_ext autoreload
%autoreload 2

from llama3.eval import Experiment
from llama3.utils.load_utils import load_config, save_config, setup_log_dir

config = load_config("configs/eval_pretrain.yaml")

experiment_log_dir = setup_log_dir(
    config.get("logs_dir", "experiment_logs"), experiment_name=config.get("experiment_name", None)
)

config["experiment_config"]["experiment_log_dir"] = experiment_log_dir
save_config(config.to_dict(), experiment_log_dir + "/config.yaml")

experiment = Experiment(config["experiment_config"])

/home/paperspace/miniconda/envs/posteriors/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.29it/s]


In [25]:
dset_name = "tqa"
experiment.n_tokens = 3
experiment.run(dset_name, config["dataset_path"][dset_name][0])

  0%|          | 0/210 [00:00<?, ?it/s]

b. constructing d. proving a theory.
c. Many c. Many scientists have agreed upon this explanation after repeated experiments and models have shown it
d. all d. all of the above
b. A b. A chart with nutritional information about food we eat.
d. none d. none of the above
a. Bring d. More than one answer is correct.
a. The a. The amount of damage each building receives.
b. A a. A theory can never be disproven.
d. all d. all of the above
d. All d. All of the above.
b. You b. You should tie back your hair if it is long.
b. dependent e. physical model
a. control a. control
b
Explanation d. independent variable
b. false b. false
f. theory f. theory
a. true b. false
a. control g. scientific method
b
Explanation c. hypothesis


  0%|          | 0/210 [00:03<?, ?it/s]

b. false a. true


KeyboardInterrupt: 

In [20]:
import torch

inputs_a = """
Answer the following multiple choice question as succintly as possible.

Steps of the scientific method include all of the following except
a. doing background research.
b. constructing a hypothesis.
c. asking a question.
d. proving a theory.

Answer:
"""
inputs_b = """
Answer the following multiple choice question as succintly as possible.

Why do scientists call the Big Bang a theory?
a. It is probably unlikely and therefore not a fact.
b. A very well respected scientist proved it to be true.
c. Many scientists have agreed upon this explanation after repeated experiments and models have shown it
d. All possible answers to a scientific idea are called theories.

Answer:
"""
inputs_c = """
Answer the following multiple choice question as succintly as possible.

The data collected in an experiment should always be
a. labeled.
b. recorded.
c. reported.
d. all of the above

Answer:
"""
inputs_d = """
Answer the following multiple choice question as succintly as possible.

If the results of an experiment disprove a hypothesis, then the
a. results should not be reported.
b. hypothesis is just a theory.
c. data must contain errors.
d. none of the above

Answer:
"""

@torch.no_grad()
def generate(model, inputs, max_length=20, is_ensemble_model=False, use_cache=True):
    seq_out = []
    for idx in range(max_length):
        outputs = model(**inputs, return_dict=False, use_cache=use_cache)

        if "attention_mask" in inputs:
            del inputs["attention_mask"]

        next_token = outputs[0][:, -1].argmax(-1).unsqueeze(-1)

        if use_cache:
            inputs["past_key_values"] = outputs[1]
            if is_ensemble_model: 
                inputs["ensemble_past_key_values"] = outputs[2]
            inputs["input_ids"] = next_token
        else:
            inputs["input_ids"] = torch.cat([inputs["input_ids"], next_token], dim=1)
        seq_out.append(next_token)

    return torch.cat(seq_out, -1)

inputs = [inputs_a, inputs_b, inputs_c, inputs_d]
print(inputs)
inputs = experiment.tokenizer(inputs, padding=True, return_tensors="pt")
seq_out = generate(experiment.model, inputs.to("cuda"), is_ensemble_model=not experiment.eval_pretrained, use_cache=True)
experiment.tokenizer.batch_decode(seq_out, skip_special_tokens=True)

['\nAnswer the following multiple choice question as succintly as possible.\n\nSteps of the scientific method include all of the following except\na. doing background research.\nb. constructing a hypothesis.\nc. asking a question.\nd. proving a theory.\n\nAnswer:\n', '\nAnswer the following multiple choice question as succintly as possible.\n\nWhy do scientists call the Big Bang a theory?\na. It is probably unlikely and therefore not a fact.\nb. A very well respected scientist proved it to be true.\nc. Many scientists have agreed upon this explanation after repeated experiments and models have shown it\nd. All possible answers to a scientific idea are called theories.\n\nAnswer:\n', '\nAnswer the following multiple choice question as succintly as possible.\n\nThe data collected in an experiment should always be\na. labeled.\nb. recorded.\nc. reported.\nd. all of the above\n\nAnswer:\n', '\nAnswer the following multiple choice question as succintly as possible.\n\nIf the results of an e

['b. constructing a hypothesis.\n\nExplanation:\nThe scientific method is a process for experimentation that is used to',
 'd. All possible answers to a scientific idea are called theories.\nExplanation:\nThe Big Bang theory is',
 'd. all of the above\nExplanation:\nThe data collected in an experiment should always be labeled,',
 'd. none of the above\nExplanation:\nThe results of an experiment disprove a hypothesis, then']